# TP Integrador - Redes Neuronales.

Clasificación para determinar si un hongo es comestible o no teniendo en cuenta distintas caracteristicas fisicas del mismo.


data set: https://www.kaggle.com/datasets/prishasawhney/mushroom-dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import seaborn as sns

### Lectura y analisis del CSV

In [ ]:
csv = pd.read_csv('mushroom.csv')
csv.isnull().sum()

In [ ]:
#csv.fillna(csv.median(), inplace=True)
csv.describe()

Por medio del histograma podemos ver que hay valores atipicos en la columna 'gill-color'

Es por esto que se descarta un 10% de cada extremo luego de ordenar el csv por esa columna

In [ ]:
csv.corr()
csv.hist(figsize=(15,15))

In [ ]:

csv_sorted = csv.sort_values(by='gill-color')

n_rows = len(csv_sorted)

ten_percent = int(0.10 * n_rows)

csv_trimmed = csv_sorted.iloc[ten_percent:-ten_percent]

csv_trimmed.to_csv('mushroom_trimmed.csv', index=False)

csv = pd.read_csv('mushroom_trimmed.csv')

Normalizacion de los datos despues de descartar los valores atipicos

In [ ]:
csv_data = csv.drop('class', axis=1)
crop = csv['class']
csv_stats = csv_data.describe().T
csv_data_n = (csv_data - csv_stats['mean']) / csv_stats['std']
csv_n = pd.concat([csv_data_n, crop], axis=1)
csv_n.describe()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

bar_width = 0.4
x = np.arange(len(csv.columns))

ax1.bar(x, csv.describe().loc['mean'], bar_width, color='green', label='Media')
ax1.set_xlabel('Características')
ax1.set_ylabel('Valor Medio', color='green')
ax1.tick_params(axis='y', labelcolor='green')
ax1.set_xticks(x)
ax1.set_xticklabels(csv.columns, rotation=45)
ax1.set_title('Valores Medios y Desvío Estándar Antes De Normalizar')

ax2 = ax1.twinx()
ax2.plot(x, csv.describe().loc['std'], color='blue', marker='o', label='Desvío Estándar')
ax2.set_ylabel('Desvío Estándar', color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

fig.tight_layout()
fig.legend(loc='upper right')

plt.show()

plt.figure(figsize=(12, 6))
plt.bar(csv_data_n.columns, csv_data_n.describe().loc['mean'], color='red')
plt.title('Valores Medios Post Normalizar')
plt.xlabel('Características')
plt.ylabel('Valor Medio')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
sns.heatmap(csv.corr() ,annot= True)

Division de los datos para la red neuronal y para usar con Scikit Learn

Se utilizo un 70% de la muestra para entrenar el modelo

In [ ]:
X_drop = csv_n.drop(['class'],axis=1)

X = X_drop.values
Y = csv_n['class'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3)

### Forward Propagation

Se utilizaron dos capas ocultas de 20 neuronas cada una.

La funcion de activacion es ReLu y la funcion de salida es Logistic

In [ ]:


input_size = X_train.shape[1] # Número de características de entrada
hidden1_size = 20  # Tamaño de la primera capa oculta
hidden2_size = 20 # Tamaño de la segunda capa oculta
output_size = 1  # Tamaño de la capa de salida

# Pesos
w_hidden1 = np.random.rand(hidden1_size, input_size)
w_hidden2 = np.random.rand(hidden2_size, hidden1_size)
w_output = np.random.rand(output_size, hidden2_size)
# Sesgos
b_hidden1 = np.random.rand(hidden1_size, 1)
b_hidden2 = np.random.rand(hidden2_size, 1)
b_output = np.random.rand(output_size, 1)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
#softmax = lambda x: np.exp(x) / np.sum(np.exp(x), axis=0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Función que ejecuta la propagación hacia adelante de la red neuronal
def forward_prop(X):
    Z1 = w_hidden1 @ X + b_hidden1
    A1 = relu(Z1)
    Z2 = w_hidden2 @ A1 + b_hidden2
    A2 = relu(Z2)
    Z3 = w_output @ A2 + b_output
    A3 = logistic(Z3) * 21 
    return Z1, A1, Z2, A2, Z3, A3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[-1]
test_comparisons = np.equal((test_predictions >= .5).astype(int), Y_test)
accuracy = np.mean(test_comparisons.astype(int))
print("Accuracy:", accuracy)

### Backpropagation

Se configuro con 100000 iteraciones y una tasa de aprendizaje del 0.0001

In [ ]:
L = .0001  # La tasa de aprendizaje
I = 100_000 # Cant de iteraciones

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2*A3 - 2*Y
    dA3_dZ3 = d_logistic(Z3)
    dZ3_dA2 = w_output
    dZ3_dW3 = A2
    dZ3_dB3 = 1
    dA2_dZ2 = d_relu(Z2)
    dZ2_dA1 = w_hidden2
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW3 = dC_dA3 @ dA3_dZ3 @ dZ3_dW3.T

    dC_dB3 = dC_dA3 @ dA3_dZ3 * dZ3_dB3

    dC_dA2 = dC_dA3 @ dA3_dZ3 @ dZ3_dA2

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3

# Ejecutar descenso de gradiente
n = X_train.shape[0]
print(n)
for i in range(I):
    # seleccionar aleatoriamente un conjunto de datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden1 -= L * dC_dW1
    b_hidden1 -= L * dC_dB1
    w_hidden2 -= L * dC_dW2
    b_hidden2 -= L * dC_dB2
    w_output -= L * dC_dW3
    b_output -= L * dC_dB3


# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[3]
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales

print("Accuracy: ", accuracy)

### Comparacion con Scikit Learn

In [ ]:
model = MLPClassifier(
	                solver='sgd', 
                    hidden_layer_sizes=(20, 20),
                    activation='relu',
                    max_iter=100_000,
                    learning_rate_init=.0001,
                    verbose=True,
                    random_state=42,
                    tol=1e-6)

model.fit(X_train, Y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Calcular la precisión del modelo

print("Puntaje del conjunto de entrenamiento: %f" % model.score(X_train, Y_train))
print("Puntaje del conjunto de prueba: %f" % model.score(X_test, Y_test))

El modelo hecho con Scikit Learn resulto tener mucha mas exactitud que el modelo manual.

Intente buscar en la red donde se puede estar perdiendo ese ~50% de exactitud pero no logre dar con el error en el codigo.

Asumo que es un error en la parte de Backpropagation ya que en el modelo de Scikit Learn se usaron las mismas configuraciones.